In [ ]:
import torch
from LSTM import LSTM
import Dataloader

model = LSTM(hidden_dim=50)

#loss_fn = torch.nn.MSELoss(size_average=False)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())#, lr=learning_rate)

num_epochs = 5  


N = 50
N_minibatch_loss = 0.0

dataloader = Dataloader.Dataloader()

inputs,targets = dataloader.load_data('../pa4Data/train.txt')

val_inputs,val_targets = dataloader.load_data('../pa4Data/val.txt')

# Track the loss across training
total_loss = []
avg_minibatch_loss = []
val_losses = []
epochs = []
minibatches = []
val_count=0
save_state_dict=None

for epoch in range(num_epochs): 
    model.clear_hidden() # zero out hidden/memory state
    
    for i in range(len(inputs)):
        model.train()
        
        # Zero out gradient
        optimizer.zero_grad()
        
        outputs = model(inputs[i])
        
        loss = criterion(outputs, targets[i].long())
          
        # Backward pass
        loss.backward()
        
        # Update parameters
        optimizer.step()

        # Add this iteration's loss to the total_loss
        total_loss.append(loss.item())
        N_minibatch_loss += loss.item()
        
        if i % N == 0: 
            # Print the loss averaged over the last N mini-batches    
            N_minibatch_loss /= N
            print('Epoch %d, average minibatch %d loss: %.3f' %
                (epoch + 1, i, N_minibatch_loss))
            
            # Add the averaged loss over N minibatches and reset the counter
            avg_minibatch_loss.append(N_minibatch_loss)
            N_minibatch_loss = 0.0
            
            # validation
            print('...validating...')
            val_loss=0.0
            with torch.no_grad():
                for v in range(len(val_inputs)):
                    val_outputs = model(val_inputs[v])
                    val_loss += criterion(val_outputs, val_targets[v]).item()
            val_loss/=len(val_inputs)
            val_losses.append(val_loss)
            
            epochs.append(epoch+1)
            minibatches.append(i)
            
            print('Epoch %d, average validation loss: %.3f' %
                (epoch + 1, val_loss))
            
            # early stopping
            if val_loss>val_losses[-1]:
                if val_count==0:
                    save_state_dict = model.state_dict()
                val_count+=1
                if val_count==5:
                    break  
            
    print("Finished", epoch + 1, "epochs of training")

if save_state_dict:
    PATH = "./output/{}/best.pt".format(model_name)
    torch.save(save_state_dict, PATH)

    


Epoch 1, average minibatch 0 loss: 0.091
...validating...
Epoch 1, average validation loss: 4.580
Epoch 1, average minibatch 50 loss: 4.291
...validating...
Epoch 1, average validation loss: 3.917
Epoch 1, average minibatch 100 loss: 3.229
...validating...
Epoch 1, average validation loss: 4.181
Epoch 1, average minibatch 150 loss: 3.684
...validating...
Epoch 1, average validation loss: 3.789
Epoch 1, average minibatch 200 loss: 3.559
...validating...
Epoch 1, average validation loss: 3.807
Epoch 1, average minibatch 250 loss: 3.353
...validating...


In [4]:
import pandas as pd

#train_loss_df = {'epoch':epochs,'minibatch':minibatches,'loss':avg_minibatch_loss}
#val_loss_df = {'epoch':epochs,'minibatch':minibatches,'loss':val_losses}

train_loss_df = pd.DataFrame({'loss':avg_minibatch_loss})
val_loss_df = pd.DataFrame({'loss':val_losses})

model_name="lstm" + str(model.hidden_dim)
val_loss_file = "./output/{}/validation_loss.csv".format(model_name)
train_loss_file = "./output/{}/train_loss.csv".format(model_name)
scores_file = "./output/{}/scores.csv".format(model_name)

train_loss_df.to_csv(train_loss_file)
val_loss_df.to_csv(val_loss_file)

In [6]:
PATH = "./output/{}/lstm.pt".format(model_name)
torch.save(model.state_dict(), PATH)